<a href="https://colab.research.google.com/github/HariRags/EmoSense/blob/main/WIDS_week2_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import re
import nltk
nltk.download('all')
from nltk.corpus import stopwords as sw
from nltk.stem.porter import PorterStemmer as ps
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()
df=pd.read_csv('/content/drive/MyDrive/twitter_data.csv',on_bad_lines='skip')
original_df= df.copy(deep = 'true')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [2]:
df=df.drop(labels= ['ItemID','SentimentSource'], axis= 1)
df['SentimentText'] = df['SentimentText'].str.replace('[^a-zA-Z]', ' ')
df['SentimentText'] = df['SentimentText'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
df['SentimentText'] = df['SentimentText'].str.lower()
df['SentimentText'] = df['SentimentText'].str.split()
df

<ipython-input-2-14d047ffd0cd>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SentimentText'] = df['SentimentText'].str.replace('[^a-zA-Z]', ' ')
<ipython-input-2-14d047ffd0cd>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SentimentText'] = df['SentimentText'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


,Sentiment,SentimentText
0,0,"[is, so, sad, for, my, apl, friend]"
1,0,"[i, missed, the, new, moon, trailer]"
2,1,"[omg, its, already, o]"
3,0,"[omgaga, im, sooo, im, gunna, cry, i, ve, been..."
4,0,"[i, think, mi, bf, is, cheating, on, me, t, t]"
...,...,...
1578607,1,"[zzzzzz, finally, night, tweeters]"
1578608,1,"[zzzzzzz, sleep, well, people]"
1578609,0,"[zzzzzzzzzz, wait, no, i, have, homework]"
1578610,0,"[zzzzzzzzzzzzz, meh, what, am, i, doing, up, a..."


In [3]:
stopwords=sw.words('english')
def remove_stopwords(text):
    stopword_list=[i for i in text if i not in stopwords]
    return stopword_list

df['SentimentText']= df['SentimentText'].apply(lambda x: remove_stopwords(x))
df['SentimentText']= df['SentimentText'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['SentimentText']

0                                         [sad, apl, friend]
1                               [missed, new, moon, trailer]
2                                             [omg, already]
3          [omgaga, im, sooo, im, gunna, cry, dentist, si...
4                                  [think, mi, bf, cheating]
                                 ...                        
1578607                    [zzzzzz, finally, night, tweeter]
1578608                       [zzzzzzz, sleep, well, people]
1578609                         [zzzzzzzzzz, wait, homework]
1578610                                 [zzzzzzzzzzzzz, meh]
1578611                          [zzzzzzzzzzzzzzzzzzz, wish]
Name: SentimentText, Length: 1578612, dtype: object

In [4]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import *
from keras.preprocessing.text import *

from keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
from keras.utils import pad_sequences

In [6]:

lb=LabelEncoder()
df['Sentiment'] = lb.fit_transform(df['Sentiment'])
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(df['SentimentText'].values)
X = tokenizer.texts_to_sequences(df['SentimentText'].values)
X = pad_sequences(X)
X

array([[  0,   0,   0, ...,   0,  46,  58],
       [  0,   0,   0, ...,   0, 171,  29],
       [  0,   0,   0, ...,   0, 147, 119],
       ...,
       [  0,   0,   0, ...,   0,   0,  70],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,  45]], dtype=int32)

In [7]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(80, dropout=0.2, recurrent_dropout=0.0))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 46, 120)           60000     
                                                                 
 spatial_dropout1d (Spatial  (None, 46, 120)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 80)                64320     
                                                                 
 dense (Dense)               (None, 2)                 162       
                                                                 
Total params: 124482 (486.26 KB)
Trainable params: 124482 (486.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
y=pd.get_dummies(df['Sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 2)
batch_size=32
model.fit(X_train, y_train, epochs = 1, batch_size=batch_size, verbose = 'auto')

34533/34533 [==============================] - 259s 7ms/step - loss: 0.5399 - accuracy: 0.7186


In [9]:
model.evaluate(X_test,y_test)

14800/14800 [==============================] - 66s 4ms/step - loss: 0.5320 - accuracy: 0.7238


[0.5319701433181763, 0.723766028881073]